In [94]:
import pandas as pd
import re
import textwrap
from common.evaluate_helpers import preprocess_text as preprocess_text_bm25
from nltk.tokenize import word_tokenize
from tabulate import tabulate

# Set display options to show full content of each column
pd.set_option("display.max_colwidth", None)


def preprocess_text(text: str) -> str:
    if type(text) != str:
        return ""

    # Remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # Remove unnecessary whitespaces
    text = re.sub(r"\s+", " ", text).strip()

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words and stem
    return " ".join([word for word in tokens])

In [92]:
MINISTRY = "minbzk"

woo_data = pd.read_csv(f"./docs_ministries_full/{MINISTRY}/woo_merged.csv.gz")

### Display all corresponding documents

In [100]:
DOSSIER = "nl.mnre1034.2i.2021.69"
TYPES = ["all"]
# TYPES = ["verzoek", "besluit"]
# TYPES = ["bijlage", "inventaris"]
RELEVANT_COLUMNS = ["dossier_id", "type", "source"]
RELEVANT_DISPLAY_COLUMNS = ["dossier_id", "document_id", "type", "source"]


if "all" in TYPES:
    filtered_data = woo_data[(woo_data["dossier_id"] == DOSSIER)]
else:
    filtered_data = woo_data[(woo_data["type"].isin(TYPES)) & (woo_data["dossier_id"] == DOSSIER)]

# Explicitly copy the filtered DataFrame to avoid SettingWithCopyWarning when modifying it.
filtered_data = filtered_data[RELEVANT_DISPLAY_COLUMNS].copy()
filtered_data.drop_duplicates(inplace=True, subset=RELEVANT_COLUMNS)
filtered_data.reset_index(inplace=True, drop=True)

print(tabulate(filtered_data, headers=RELEVANT_DISPLAY_COLUMNS, tablefmt="psql"))

+----+------------------------+--------------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | dossier_id             | document_id                    | type    | source                                                                                                                                                                      |
|----+------------------------+--------------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | nl.mnre1034.2i.2021.69 | nl.mnre1034.2i.2021.69.misc.1  | besluit | https://open.overheid.nl/repository/ronl-9979d8d4-3bae-4b5b-96c9-048ead254006/1/pdf/Besluit%20Wob-verzoek%20over%20vuurwerkramp%20Enschede.pdf                              |
|  1

### Display the bodytext of a page

In [99]:
PAGE = "nl.mnre1034.2i.2021.69.misc.5.pagina.13"
RELEVANT_DISPLAY_COLUMNS = ["bodyText"]


filtered_data = woo_data[(woo_data["page_id"] == PAGE)]
filtered_data = filtered_data[RELEVANT_DISPLAY_COLUMNS].copy()
filtered_data.reset_index(inplace=True, drop=True)

all_text = " ".join(filtered_data["bodyText"].astype(str))
# processed_text = preprocess_text(all_text)
processed_text = " ".join(preprocess_text_bm25(all_text))

print(textwrap.fill(processed_text, width=140))

eben ni all kel ull vatum la jul uv rapport dossiermeldingactiviteit bladnr t activiteit t activiteit kao eod cee uur dcbc gee po co poo kgg golfci lost deltaci is eigen mensen deltaci gebeurd de peloton geregeld lost peloton me politi deltaci uur uur bey cod cap ee nh meldt me geregeld kennemerland ievert om uur ís geschakeld belangrijk bij overdracht en verder op te pakken extra peloton zijn geregeld zh levert heden van tot uur nh morgen vanaf uur tot uur let op atw bij bijstand me vanuit zh en nh commentaar gekregen dcbc wil weten hoe nu verder in de komend dagen pcc overijssel wil graag schriftelijk bevestig dat de extra me peloton politi worden en geen kmar iei uit ge laamem wil schriftelijk sitrap graag toegefaxt krijgen bez eel nh vraagt of de verbindingscommandoauto ook mee moet omdat gevraagd wordt eigen vebindigsnmiddelen mee te nemen nog niet nagevraagd bij overijssel be vom kan nog terugbellen svp informati meegeven van meld uur me ct in telefoon diversen kkrrrkraaaraakraar

### Display the bodytext of a document

In [78]:
DOCUMENT = "nl.mnre1058.2i.2018.37.doc.2"
RELEVANT_DISPLAY_COLUMNS = ["bodyText"]


filtered_data = woo_data[(woo_data["document_id"] == DOCUMENT)]
filtered_data = filtered_data[RELEVANT_DISPLAY_COLUMNS].copy()
filtered_data.reset_index(inplace=True, drop=True)

# Apply preprocessing to each text entry
filtered_data["bodyText"] = filtered_data["bodyText"].apply(preprocess_text)

all_text = " ".join(filtered_data["bodyText"])

print(textwrap.fill(all_text, width=140))

Dienst Justitiële Inrichtingen Ministerie van Veiligheid en Justitie Retouradres Postbus 30132 2500 GC Den Haag Directie Beleid
Bestuursondersteuning Juridische Zaken ADR Nieuwmedia Turfmarkt 147 2511 DP Den Haag Postbus 30132 2500 CC Den Haag www dj 1 fl1
Contactpersoon Alleen oer email Ons kenmerk 2164748 Datum 9 mei 2018 Bijlage Onderwerp Uw Wob verzoek om inzage in
medewerkerstevredenheidsonderzoeken Bq beantwoording de datum en ons kenmerk vermelden Wilt u slechts één zaak in uw brief behandelen
Geachte heel__________________ Bij brief van 29 november 2017 door mij ontvangen op 1 december 2017 verzoekt u op grond van de Wet
openbaarheid bestuur de Wob om inzage in alle gerapporteerde medewerkerstvredenheidsonderzoeken hierna MTOs die zijn gehouden onder de
medewerkers van de Pls Haaglanden en Pl Vught in de jaren 2016 en 2017 Bij brief van 8 december 2017 heb ik de ontvangst van uw verzoek
bevestigd en bij brief van 22 december 2017 heb ik het besluit op uw verzoek verdaagd met vi